In [1]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re

In [2]:
#change path
path="..\\keystroke.csv"

In [3]:
import pandas
import numpy as np
data = pandas.read_csv(path)
subjects = data["subject"].unique()

In [4]:
subjects

array(['s002', 's003', 's004', 's005', 's007', 's008', 's010', 's011',
       's012', 's013', 's015', 's016', 's017', 's018', 's019', 's020',
       's021', 's022', 's024', 's025', 's026', 's027', 's028', 's029',
       's030', 's031', 's032', 's033', 's034', 's035', 's036', 's037',
       's038', 's039', 's040', 's041', 's042', 's043', 's044', 's046',
       's047', 's048', 's049', 's050', 's051', 's052', 's053', 's054',
       's055', 's056', 's057'], dtype=object)

In [5]:

mean_vector = []

In [6]:
from sklearn.mixture import GMM
import pandas 
#from EER_GMM import evaluateEER
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [7]:
def training(train):
    gmm = GMM(n_components = 2, covariance_type = 'diag', 
                    verbose = False )
    gmm.fit(train)
    return gmm

In [8]:
def testing(test_genuine, test_imposter, gmm):
    user_scores = []
    imposter_scores = []
    for i in range(test_genuine.shape[0]):
        j = test_genuine.iloc[i].values
        cur_score = gmm.score([j])
        user_scores.append(cur_score)

    for i in range(test_imposter.shape[0]):
        j = test_imposter.iloc[i].values
        cur_score = gmm.score([j])
        imposter_scores.append(cur_score)
        
    return user_scores, imposter_scores

In [32]:
def evaluateEERGMM(user_scores, imposter_scores):
    thresholds = range(20,51)
    array = np.zeros((len(thresholds),3))
    i = 0
    for th in thresholds:
        g_i = 0
        i_g = 0
        for score in user_scores:
            if score < th:
                g_i = g_i + 1
        for score in imposter_scores:    
            if score > th:
                i_g = i_g + 1

        
        FR = float(g_i) / len(user_scores)
        FA = float(i_g) / len(imposter_scores) 
        
        array[i, 0] = th
        array[i, 1] = FA
        array[i, 2] = FR
        i = i + 1
    print(array)
    for j in range(array.shape[0]):
        if array[j,1] < array[j,2]:
            thresh = (array[j,0] + array[j - 1, 0]) / 2
            break
    print(thresh)
    g_i = 0
    i_g = 0
    for score in user_scores:
        if score < thresh:
            g_i = g_i + 1
    for score in imposter_scores:    
        if score > thresh:
            i_g = i_g + 1

    FA = float(i_g) / len(imposter_scores) 
    FR = float(g_i) / len(user_scores)
    return (FA + FR) /2
    

In [33]:
def evaluate(subjects):    
    eers = []

    for subject in subjects:        
        genuine_user_data = data.loc[data.subject == subject, \
                                     "H.period":"H.Return"]
        imposter_data = data.loc[data.subject != subject, :]

        train = genuine_user_data[:200]
        test_genuine = genuine_user_data[200:]
        test_imposter = imposter_data.groupby("subject"). \
                             head(5).loc[:, "H.period":"H.Return"]

        gmm = training(train)
        user_scores, imposter_scores = testing(test_genuine, test_imposter, gmm)
        eers.append(evaluateEERGMM(user_scores, imposter_scores))
    return np.mean(eers)

In [34]:
print ("average EER for GMM detector:")

print(evaluate(subjects))

average EER for GMM detector:
[[2.00e+01 5.36e-01 2.00e-02]
 [2.10e+01 5.20e-01 2.00e-02]
 [2.20e+01 5.16e-01 2.00e-02]
 [2.30e+01 5.12e-01 2.00e-02]
 [2.40e+01 5.00e-01 2.50e-02]
 [2.50e+01 4.88e-01 2.50e-02]
 [2.60e+01 4.88e-01 2.50e-02]
 [2.70e+01 4.80e-01 3.50e-02]
 [2.80e+01 4.72e-01 4.50e-02]
 [2.90e+01 4.60e-01 4.50e-02]
 [3.00e+01 4.48e-01 5.50e-02]
 [3.10e+01 4.44e-01 5.50e-02]
 [3.20e+01 4.24e-01 6.50e-02]
 [3.30e+01 4.08e-01 8.00e-02]
 [3.40e+01 3.88e-01 9.00e-02]
 [3.50e+01 3.68e-01 1.00e-01]
 [3.60e+01 3.36e-01 1.30e-01]
 [3.70e+01 3.20e-01 1.45e-01]
 [3.80e+01 2.84e-01 1.45e-01]
 [3.90e+01 2.72e-01 1.65e-01]
 [4.00e+01 2.52e-01 1.90e-01]
 [4.10e+01 2.08e-01 2.15e-01]
 [4.20e+01 1.68e-01 2.25e-01]
 [4.30e+01 1.52e-01 2.25e-01]
 [4.40e+01 1.24e-01 2.35e-01]
 [4.50e+01 1.12e-01 2.60e-01]
 [4.60e+01 9.20e-02 2.75e-01]
 [4.70e+01 8.80e-02 2.95e-01]
 [4.80e+01 7.60e-02 3.10e-01]
 [4.90e+01 7.20e-02 3.30e-01]
 [5.00e+01 6.80e-02 3.50e-01]]
40.5
[[2.00e+01 3.88e-01 2.50e-02]
 [2.

[[2.00e+01 3.36e-01 4.50e-02]
 [2.10e+01 3.36e-01 5.00e-02]
 [2.20e+01 3.32e-01 5.00e-02]
 [2.30e+01 3.20e-01 5.00e-02]
 [2.40e+01 3.08e-01 5.00e-02]
 [2.50e+01 2.96e-01 5.00e-02]
 [2.60e+01 2.92e-01 5.00e-02]
 [2.70e+01 2.84e-01 5.00e-02]
 [2.80e+01 2.72e-01 5.00e-02]
 [2.90e+01 2.72e-01 5.00e-02]
 [3.00e+01 2.64e-01 5.00e-02]
 [3.10e+01 2.52e-01 5.50e-02]
 [3.20e+01 2.40e-01 5.50e-02]
 [3.30e+01 2.32e-01 6.00e-02]
 [3.40e+01 2.24e-01 6.50e-02]
 [3.50e+01 2.08e-01 6.50e-02]
 [3.60e+01 2.04e-01 7.00e-02]
 [3.70e+01 1.84e-01 7.00e-02]
 [3.80e+01 1.80e-01 7.50e-02]
 [3.90e+01 1.64e-01 8.50e-02]
 [4.00e+01 1.52e-01 9.50e-02]
 [4.10e+01 1.40e-01 1.10e-01]
 [4.20e+01 1.36e-01 1.20e-01]
 [4.30e+01 1.24e-01 1.30e-01]
 [4.40e+01 1.00e-01 1.45e-01]
 [4.50e+01 8.40e-02 1.50e-01]
 [4.60e+01 7.60e-02 1.70e-01]
 [4.70e+01 7.20e-02 1.90e-01]
 [4.80e+01 5.20e-02 1.95e-01]
 [4.90e+01 4.00e-02 2.15e-01]
 [5.00e+01 2.80e-02 2.20e-01]]
42.5
[[2.00e+01 4.44e-01 2.50e-02]
 [2.10e+01 4.36e-01 2.50e-02]
 [2.

[[2.00e+01 4.92e-01 2.50e-02]
 [2.10e+01 4.76e-01 2.50e-02]
 [2.20e+01 4.68e-01 2.50e-02]
 [2.30e+01 4.64e-01 2.50e-02]
 [2.40e+01 4.56e-01 2.50e-02]
 [2.50e+01 4.44e-01 2.50e-02]
 [2.60e+01 4.36e-01 3.00e-02]
 [2.70e+01 4.36e-01 3.50e-02]
 [2.80e+01 4.20e-01 3.50e-02]
 [2.90e+01 4.04e-01 3.50e-02]
 [3.00e+01 3.80e-01 3.50e-02]
 [3.10e+01 3.64e-01 4.00e-02]
 [3.20e+01 3.56e-01 4.50e-02]
 [3.30e+01 3.24e-01 4.50e-02]
 [3.40e+01 3.12e-01 5.00e-02]
 [3.50e+01 2.96e-01 6.00e-02]
 [3.60e+01 2.60e-01 6.00e-02]
 [3.70e+01 2.36e-01 6.00e-02]
 [3.80e+01 2.16e-01 7.00e-02]
 [3.90e+01 1.92e-01 8.00e-02]
 [4.00e+01 1.84e-01 8.50e-02]
 [4.10e+01 1.56e-01 8.50e-02]
 [4.20e+01 1.48e-01 1.00e-01]
 [4.30e+01 1.28e-01 1.25e-01]
 [4.40e+01 1.04e-01 1.70e-01]
 [4.50e+01 8.40e-02 2.15e-01]
 [4.60e+01 6.40e-02 2.30e-01]
 [4.70e+01 5.60e-02 2.60e-01]
 [4.80e+01 3.60e-02 2.70e-01]
 [4.90e+01 2.00e-02 2.80e-01]
 [5.00e+01 2.00e-02 3.20e-01]]
43.5
[[2.00e+01 5.96e-01 2.50e-02]
 [2.10e+01 5.84e-01 2.50e-02]
 [2.

[[2.00e+01 4.04e-01 7.00e-02]
 [2.10e+01 3.68e-01 9.50e-02]
 [2.20e+01 3.20e-01 1.20e-01]
 [2.30e+01 2.96e-01 1.35e-01]
 [2.40e+01 2.80e-01 1.60e-01]
 [2.50e+01 2.60e-01 1.80e-01]
 [2.60e+01 1.96e-01 2.30e-01]
 [2.70e+01 1.80e-01 2.50e-01]
 [2.80e+01 1.60e-01 2.60e-01]
 [2.90e+01 1.44e-01 2.80e-01]
 [3.00e+01 1.24e-01 3.10e-01]
 [3.10e+01 1.00e-01 3.30e-01]
 [3.20e+01 6.40e-02 3.45e-01]
 [3.30e+01 5.20e-02 3.60e-01]
 [3.40e+01 4.00e-02 4.10e-01]
 [3.50e+01 2.80e-02 4.55e-01]
 [3.60e+01 1.60e-02 5.10e-01]
 [3.70e+01 1.60e-02 5.50e-01]
 [3.80e+01 1.20e-02 5.90e-01]
 [3.90e+01 4.00e-03 6.40e-01]
 [4.00e+01 4.00e-03 7.65e-01]
 [4.10e+01 0.00e+00 8.30e-01]
 [4.20e+01 0.00e+00 9.00e-01]
 [4.30e+01 0.00e+00 9.60e-01]
 [4.40e+01 0.00e+00 9.75e-01]
 [4.50e+01 0.00e+00 9.95e-01]
 [4.60e+01 0.00e+00 1.00e+00]
 [4.70e+01 0.00e+00 1.00e+00]
 [4.80e+01 0.00e+00 1.00e+00]
 [4.90e+01 0.00e+00 1.00e+00]
 [5.00e+01 0.00e+00 1.00e+00]]
25.5
[[2.00e+01 5.64e-01 4.50e-02]
 [2.10e+01 5.48e-01 5.00e-02]
 [2.

[[2.00e+01 5.08e-01 2.00e-02]
 [2.10e+01 4.96e-01 2.50e-02]
 [2.20e+01 4.88e-01 2.50e-02]
 [2.30e+01 4.76e-01 2.50e-02]
 [2.40e+01 4.60e-01 2.50e-02]
 [2.50e+01 4.44e-01 2.50e-02]
 [2.60e+01 4.40e-01 2.50e-02]
 [2.70e+01 4.08e-01 2.50e-02]
 [2.80e+01 3.96e-01 2.50e-02]
 [2.90e+01 3.76e-01 3.00e-02]
 [3.00e+01 3.60e-01 3.50e-02]
 [3.10e+01 3.40e-01 3.50e-02]
 [3.20e+01 3.20e-01 4.00e-02]
 [3.30e+01 2.88e-01 4.50e-02]
 [3.40e+01 2.80e-01 5.00e-02]
 [3.50e+01 2.20e-01 5.50e-02]
 [3.60e+01 1.92e-01 5.50e-02]
 [3.70e+01 1.72e-01 6.00e-02]
 [3.80e+01 1.32e-01 6.00e-02]
 [3.90e+01 1.16e-01 6.50e-02]
 [4.00e+01 8.40e-02 6.50e-02]
 [4.10e+01 5.20e-02 6.50e-02]
 [4.20e+01 4.40e-02 6.50e-02]
 [4.30e+01 3.60e-02 6.50e-02]
 [4.40e+01 2.40e-02 7.50e-02]
 [4.50e+01 2.40e-02 8.50e-02]
 [4.60e+01 2.00e-02 8.50e-02]
 [4.70e+01 2.00e-02 8.50e-02]
 [4.80e+01 8.00e-03 9.00e-02]
 [4.90e+01 8.00e-03 9.00e-02]
 [5.00e+01 4.00e-03 1.00e-01]]
40.5
[[2.00e+01 5.24e-01 2.50e-02]
 [2.10e+01 5.04e-01 3.00e-02]
 [2.

[[2.00e+01 2.28e-01 2.00e-02]
 [2.10e+01 2.04e-01 2.00e-02]
 [2.20e+01 1.88e-01 2.00e-02]
 [2.30e+01 1.64e-01 2.00e-02]
 [2.40e+01 1.44e-01 2.00e-02]
 [2.50e+01 1.12e-01 2.00e-02]
 [2.60e+01 1.04e-01 2.00e-02]
 [2.70e+01 9.60e-02 2.00e-02]
 [2.80e+01 8.40e-02 2.50e-02]
 [2.90e+01 5.20e-02 2.50e-02]
 [3.00e+01 4.00e-02 2.50e-02]
 [3.10e+01 2.40e-02 2.50e-02]
 [3.20e+01 1.20e-02 3.50e-02]
 [3.30e+01 4.00e-03 4.00e-02]
 [3.40e+01 4.00e-03 4.00e-02]
 [3.50e+01 4.00e-03 4.00e-02]
 [3.60e+01 4.00e-03 4.50e-02]
 [3.70e+01 0.00e+00 4.50e-02]
 [3.80e+01 0.00e+00 4.50e-02]
 [3.90e+01 0.00e+00 5.00e-02]
 [4.00e+01 0.00e+00 5.00e-02]
 [4.10e+01 0.00e+00 5.50e-02]
 [4.20e+01 0.00e+00 7.00e-02]
 [4.30e+01 0.00e+00 8.00e-02]
 [4.40e+01 0.00e+00 8.00e-02]
 [4.50e+01 0.00e+00 8.50e-02]
 [4.60e+01 0.00e+00 9.00e-02]
 [4.70e+01 0.00e+00 9.50e-02]
 [4.80e+01 0.00e+00 1.10e-01]
 [4.90e+01 0.00e+00 1.15e-01]
 [5.00e+01 0.00e+00 1.35e-01]]
30.5
[[2.00e+01 1.88e-01 1.50e-02]
 [2.10e+01 1.80e-01 1.50e-02]
 [2.

In [12]:
subject = subjects[0]
subject


's002'

In [41]:
genuine_user_data = data.loc[data.subject == subject, \
                             "H.period":"H.Return"]
imposter_data = data.loc[data.subject != subject, :]

train = genuine_user_data[:200]
test_genuine = genuine_user_data[200:]
test_imposter = imposter_data.groupby("subject"). \
                     head(5).loc[:, "H.period":"H.Return"]

gmm = training(train)
user_scores, imposter_scores = testing(test_genuine, test_imposter, gmm)
imposter_scores

[array([43.36860091]),
 array([41.93577126]),
 array([35.30394054]),
 array([-138.63295814]),
 array([-37.8400682]),
 array([45.49788504]),
 array([33.7441285]),
 array([18.04749482]),
 array([43.65917184]),
 array([47.78350343]),
 array([-456.88710513]),
 array([-45.24212821]),
 array([-125.87022192]),
 array([-21.17157901]),
 array([-8.31715561]),
 array([28.63314349]),
 array([23.86628144]),
 array([24.90927436]),
 array([20.07122887]),
 array([40.05906835]),
 array([41.7156826]),
 array([40.25138188]),
 array([42.52625161]),
 array([40.69067002]),
 array([40.79359065]),
 array([39.26507693]),
 array([12.44810289]),
 array([35.60079248]),
 array([51.11046171]),
 array([51.64812107]),
 array([43.94746012]),
 array([53.82182446]),
 array([50.46278185]),
 array([43.05054085]),
 array([40.51064183]),
 array([29.18505221]),
 array([11.58751868]),
 array([24.96542346]),
 array([-86.8840574]),
 array([21.39391494]),
 array([31.67465599]),
 array([54.04482691]),
 array([35.09101915]),
 arra

In [13]:
user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
train_data = user_data[:200]
test_data = user_data[200:]

In [14]:
other_user_data = data.loc[data.subject != subject, "H.period":"H.Return"]
other_user_data

,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
400,0.1333,0.2636,0.1303,0.1246,0.1737,0.0491,0.1330,0.1977,0.0647,0.0934,...,0.1291,0.2234,0.0943,0.1090,0.3811,0.2721,0.1003,0.2997,0.1994,0.1035
401,0.1307,0.3850,0.2543,0.1214,0.1908,0.0694,0.0956,0.2007,0.1051,0.1267,...,0.1016,0.2365,0.1349,0.0781,0.2932,0.2151,0.1138,0.3201,0.2063,0.1035
402,0.1143,0.2752,0.1609,0.1148,0.1439,0.0291,0.1468,0.4033,0.2565,0.1504,...,0.1132,0.2104,0.0972,0.1351,0.2502,0.1151,0.1227,0.2883,0.1656,0.0800
403,0.1328,0.2296,0.0968,0.0985,0.7023,0.6038,0.1222,0.6574,0.5352,0.1423,...,0.1470,0.1824,0.0354,0.0892,0.3454,0.2562,0.1090,0.3079,0.1989,0.0958
404,0.1217,0.1863,0.0646,0.1175,0.1584,0.0409,0.1175,0.5223,0.4048,0.0824,...,0.1288,0.1421,0.0133,0.0618,0.2373,0.1755,0.0971,0.2632,0.1661,0.0987
405,0.1320,0.1979,0.0659,0.1217,0.1523,0.0306,0.0964,0.1714,0.0750,0.0945,...,0.1014,0.1542,0.0528,0.0871,0.1314,0.0443,0.1164,0.2783,0.1619,0.1008
406,0.1235,0.2285,0.1050,0.0913,0.1594,0.0681,0.0895,0.2898,0.2003,0.1557,...,0.1291,0.1183,-0.0108,0.0861,0.1331,0.0470,0.1043,0.2702,0.1659,0.1098
407,0.1190,0.3878,0.2688,0.1032,0.1291,0.0259,0.1135,0.1452,0.0317,0.1204,...,0.1006,0.1183,0.0177,0.0713,0.1581,0.0868,0.0929,0.2411,0.1482,0.0829
408,0.0921,0.2741,0.1820,0.1172,0.5906,0.4734,0.1124,0.1837,0.0713,0.1193,...,0.1109,0.1690,0.0581,0.0465,0.1114,0.0649,0.0887,0.3446,0.2559,0.0927
409,0.1283,0.6059,0.4776,0.1246,0.1442,0.0196,0.1169,0.2162,0.0993,0.1185,...,0.1048,0.1529,0.0481,0.1074,0.1823,0.0749,0.1029,0.2804,0.1775,0.0824


In [15]:
gmm = GMM(n_components = 2, covariance_type = 'diag', verbose = False )
gmm.fit(train_data)
gmm

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=2, n_init=1, n_iter=100, params='wmc', random_state=None,
  tol=0.001, verbose=False)

In [16]:
gmm.score([test_data.iloc[10].values])

array([57.26331025])

In [17]:
gmm.score([other_user_data.iloc[10].values])

array([36.54660986])

In [36]:
for i in range(10):
    print(gmm.score([other_user_data.iloc[i].values]))
    print(gmm.predict([other_user_data.iloc[i].values]))

[43.36860091]
[0]
[41.93577126]
[0]
[35.30394054]
[1]
[-138.63295814]
[1]
[-37.8400682]
[0]
[55.23040567]
[1]
[27.4139615]
[0]
[41.22433495]
[0]
[-38.19716614]
[0]
[30.89637318]
[0]


In [19]:
gmm.weights_

array([0.32911594, 0.67088406])

In [20]:
train_identification = data.groupby("subject").head(200).loc[:, "H.period":"H.Return"]

In [21]:
X_train = train_identification.as_matrix

In [22]:
gmm_identification = GMM(n_components = subjects.shape[0], covariance_type = 'diag', verbose = False )
gmm_identification.fit(train_identification)

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=51, n_init=1, n_iter=100, params='wmc', random_state=None,
  tol=0.001, verbose=False)

In [23]:
gmm_identification.weights_

array([5.09166091e-03, 1.64293046e-02, 9.80392157e-05, 4.41102327e-03,
       7.78436542e-03, 1.70628941e-02, 2.15558428e-02, 9.80392157e-05,
       3.82902707e-02, 1.31891883e-02, 1.44822270e-02, 3.80854673e-02,
       1.78683716e-02, 4.74688521e-03, 9.30962265e-03, 4.48504295e-03,
       4.54855819e-02, 1.77351438e-02, 1.22084620e-03, 8.13927872e-02,
       3.35282574e-03, 9.80392157e-05, 1.75075293e-02, 5.17903754e-02,
       9.86852134e-03, 7.40835089e-03, 1.84381953e-02, 2.94117647e-04,
       1.96078431e-04, 6.81058676e-03, 8.70064299e-03, 4.54588974e-03,
       1.04829008e-01, 3.21307290e-02, 9.31387382e-03, 2.94637000e-03,
       8.12217504e-03, 1.85013081e-01, 1.90554654e-02, 1.46006165e-02,
       3.08928975e-02, 1.11820415e-02, 1.15804304e-02, 1.60914560e-03,
       1.57677313e-02, 1.42000028e-02, 2.45722675e-03, 3.92927853e-04,
       7.86977814e-03, 1.67400156e-02, 2.34627266e-02])

In [24]:
test_identification = data.groupby("subject").tail(100).loc[:, "H.period":"H.Return"]

In [25]:
test_identification

,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
300,0.1016,0.2870,0.1854,0.0718,0.1214,0.0496,0.0576,0.1252,0.0676,0.0810,...,0.1096,0.0864,-0.0232,0.0800,0.1993,0.1193,0.0845,0.2359,0.1514,0.0766
301,0.1095,0.2002,0.0907,0.0692,0.1012,0.0320,0.0763,0.1056,0.0293,0.0768,...,0.1124,0.1164,0.0040,0.0787,0.1653,0.0866,0.0774,0.1855,0.1081,0.0750
302,0.1029,0.1390,0.0361,0.0694,0.1043,0.0349,0.0723,0.0977,0.0254,0.0787,...,0.1017,0.0550,-0.0467,0.0850,0.1942,0.1092,0.0721,0.1886,0.1165,0.0684
303,0.1093,0.1193,0.0100,0.0702,0.1267,0.0565,0.0776,0.1014,0.0238,0.0710,...,0.0869,0.0755,-0.0114,0.0777,0.1650,0.0873,0.0829,0.1749,0.0920,0.0718
304,0.0979,0.1519,0.0540,0.0721,0.1022,0.0301,0.0721,0.1112,0.0391,0.0781,...,0.1193,0.0837,-0.0356,0.0977,0.1755,0.0778,0.1019,0.2079,0.1060,0.0792
305,0.0919,0.0998,0.0079,0.0911,0.1328,0.0417,0.0718,0.0943,0.0225,0.0876,...,0.1028,0.0922,-0.0106,0.1041,0.1965,0.0924,0.0810,0.1936,0.1126,0.0734
306,0.0934,0.1224,0.0290,0.0671,0.1154,0.0483,0.0774,0.0912,0.0138,0.0713,...,0.0821,0.1452,0.0631,0.0618,0.1623,0.1005,0.0847,0.1822,0.0975,0.0803
307,0.0927,0.1135,0.0208,0.0665,0.1283,0.0618,0.0702,0.1096,0.0394,0.0842,...,0.1206,0.4057,0.2851,0.1320,0.2946,0.1626,0.1003,0.2374,0.1371,0.0832
308,0.1188,0.2090,0.0902,0.0668,0.0930,0.0262,0.0866,0.1410,0.0544,0.0781,...,0.0993,0.0827,-0.0166,0.0776,0.1668,0.0892,0.0871,0.2195,0.1324,0.0821
309,0.1161,0.1828,0.0667,0.0628,0.0887,0.0259,0.0964,0.1276,0.0312,0.0713,...,0.1267,0.0573,-0.0694,0.0871,0.1879,0.1008,0.0927,0.2193,0.1266,0.1035


In [26]:
gmm.score(test_identification)

array([56.31532553, 43.98916687, 58.02713669, ..., 43.71979752,
       37.76612089, 40.2257287 ])

In [27]:
predictions = gmm.predict([test_identification.iloc[500].values])

In [28]:
predictions

array([1], dtype=int64)